# White Toroidal Warp Drive

This notebook explores the toroidal warp drive configuration.

**Reference**: White, H. (2011). AIAA SPACE Conference.

**[ASSUMPTION]**: This implementation is an interpretation of White's AIAA presentations. The exact published metric form may differ from this implementation.

## Key Concept

White proposed concentrating the exotic matter in a **toroidal (donut) shape** rather than a spherical shell. This potentially reduces total energy requirements by optimizing the geometry.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from warpbubblesim.metrics import AlcubierreMetric, WhiteToroidalMetric
from warpbubblesim.gr import compute_energy_density
from warpbubblesim.viz.fields2d import plot_energy_density, plot_field_2d

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

## 1. Toroidal Geometry

A torus is defined by:
- **Major radius** $R_{major}$: Distance from center to tube center
- **Minor radius** $R_{minor}$: Tube radius

The shape function is based on distance from the torus surface.

In [ ]:
# Create White toroidal metric
white = WhiteToroidalMetric(
    v0=1.0,
    R_major=2.0,  # Distance from center to tube
    R_minor=0.5,  # Tube radius
    sigma=8.0
)

info = white.info()
print(f"Metric: {info['name']}")
print(f"Citation: {info['citation']}")
print(f"Note: {info.get('note', 'N/A')}")
print(f"Parameters: {info['parameters']}")

## 2. Torus Visualization

In [ ]:
# 3D visualization of the torus
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

R_major = 2.0
R_minor = 0.5

# Parametric torus
u = np.linspace(0, 2*np.pi, 50)
v = np.linspace(0, 2*np.pi, 30)
U, V = np.meshgrid(u, v)

# The torus is oriented around the x-axis for warp drive
X = (R_major + R_minor * np.cos(V)) * np.cos(U)  # This would be y in our coords
Y = (R_major + R_minor * np.cos(V)) * np.sin(U)  # This would be z in our coords
Z = R_minor * np.sin(V)  # This is x deviation from center

ax.plot_surface(Z, X, Y, alpha=0.5, cmap='viridis')
ax.set_xlabel('x (direction of travel)')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title(f'Toroidal Warp Field\n$R_{{major}}={R_major}$, $R_{{minor}}={R_minor}$')

# Add travel direction arrow
ax.quiver(-3, 0, 0, 6, 0, 0, color='red', arrow_length_ratio=0.1, linewidth=2)

plt.tight_layout()
plt.show()

## 3. Shape Function Comparison

In [ ]:
# Compare shape functions: Alcubierre (spherical) vs White (toroidal)
alc = AlcubierreMetric(v0=1.0, R=1.0)
white = WhiteToroidalMetric(v0=1.0, R_major=1.5, R_minor=0.5)

# Plot in the y=0 plane
x = np.linspace(-3, 3, 100)
z = np.linspace(-3, 3, 100)
X, Z = np.meshgrid(x, z)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Alcubierre shape function (at y=0)
F_alc = np.zeros_like(X)
for i in range(len(z)):
    for j in range(len(x)):
        r_s = alc.r_from_center(0, x[j], 0, z[i])
        F_alc[i, j] = alc.shape_function(r_s)

im = axes[0].contourf(X, Z, F_alc, levels=50, cmap='viridis')
plt.colorbar(im, ax=axes[0])
axes[0].set_xlabel('x')
axes[0].set_ylabel('z')
axes[0].set_title('Alcubierre (Spherical)')
axes[0].set_aspect('equal')

# White toroidal shape function (at y=0)
F_white = np.zeros_like(X)
for i in range(len(z)):
    for j in range(len(x)):
        F_white[i, j] = white.toroidal_shape_function(0, x[j], 0, z[i])

im = axes[1].contourf(X, Z, F_white, levels=50, cmap='viridis')
plt.colorbar(im, ax=axes[1])
axes[1].set_xlabel('x')
axes[1].set_ylabel('z')
axes[1].set_title('White (Toroidal) - y=0 slice')
axes[1].set_aspect('equal')

plt.tight_layout()
plt.show()

## 4. Energy Density Distribution

In [ ]:
# Compare energy density
alc = AlcubierreMetric(v0=1.0, R=1.5)
white = WhiteToroidalMetric(v0=1.0, R_major=1.5, R_minor=0.5)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

plot_energy_density(alc, x_range=(-3, 3), y_range=(-3, 3), 
                    nx=80, ny=80, ax=axes[0])
axes[0].set_title('Alcubierre Energy Density')

plot_energy_density(white, x_range=(-3, 3), y_range=(-3, 3), 
                    nx=80, ny=80, ax=axes[1])
axes[1].set_title('White Toroidal Energy Density')

plt.tight_layout()
plt.show()

## 5. Different Cross-Sections

In [ ]:
# View energy density in different planes
white = WhiteToroidalMetric(v0=1.0, R_major=2.0, R_minor=0.5)
metric_func = white.get_metric_func()

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# x-y plane (z=0)
x = np.linspace(-3, 3, 80)
y = np.linspace(-3, 3, 80)
X, Y = np.meshgrid(x, y)
rho_xy = np.zeros_like(X)
for i in range(len(y)):
    for j in range(len(x)):
        rho_xy[i, j] = compute_energy_density(metric_func, np.array([0, x[j], y[i], 0]))

vmax = np.max(np.abs(rho_xy))
im = axes[0].contourf(X, Y, rho_xy, levels=50, cmap='RdBu_r', vmin=-vmax, vmax=vmax)
plt.colorbar(im, ax=axes[0])
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('x-y plane (z=0)')
axes[0].set_aspect('equal')

# x-z plane (y=0)
z = np.linspace(-3, 3, 80)
X, Z = np.meshgrid(x, z)
rho_xz = np.zeros_like(X)
for i in range(len(z)):
    for j in range(len(x)):
        rho_xz[i, j] = compute_energy_density(metric_func, np.array([0, x[j], 0, z[i]]))

vmax = np.max(np.abs(rho_xz))
im = axes[1].contourf(X, Z, rho_xz, levels=50, cmap='RdBu_r', vmin=-vmax, vmax=vmax)
plt.colorbar(im, ax=axes[1])
axes[1].set_xlabel('x')
axes[1].set_ylabel('z')
axes[1].set_title('x-z plane (y=0)')
axes[1].set_aspect('equal')

# y-z plane (x=0)
Y, Z = np.meshgrid(y, z)
rho_yz = np.zeros_like(Y)
for i in range(len(z)):
    for j in range(len(y)):
        rho_yz[i, j] = compute_energy_density(metric_func, np.array([0, 0, y[j], z[i]]))

vmax = np.max(np.abs(rho_yz))
im = axes[2].contourf(Y, Z, rho_yz, levels=50, cmap='RdBu_r', vmin=-vmax, vmax=vmax)
plt.colorbar(im, ax=axes[2])
axes[2].set_xlabel('y')
axes[2].set_ylabel('z')
axes[2].set_title('y-z plane (x=0) - Torus cross-section')
axes[2].set_aspect('equal')

plt.tight_layout()
plt.show()

## 6. Parameter Study

In [ ]:
# Effect of aspect ratio
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

configs = [
    (2.0, 0.3, 'Thin torus'),
    (2.0, 0.7, 'Medium torus'),
    (1.5, 0.5, 'Compact torus'),
    (3.0, 0.5, 'Wide torus'),
]

for ax, (R_major, R_minor, name) in zip(axes, configs):
    metric = WhiteToroidalMetric(v0=1.0, R_major=R_major, R_minor=R_minor)
    plot_energy_density(metric, x_range=(-4, 4), y_range=(-4, 4), 
                        nx=60, ny=60, ax=ax)
    ax.set_title(f'{name}\n$R_{{major}}={R_major}$, $R_{{minor}}={R_minor}$')

plt.tight_layout()
plt.show()

## 7. Torus Properties

In [ ]:
print("Torus Properties:")
print("="*50)

for R_major, R_minor in [(2.0, 0.5), (3.0, 0.5), (2.0, 1.0)]:
    metric = WhiteToroidalMetric(v0=1.0, R_major=R_major, R_minor=R_minor)
    params = metric.torus_parameters()
    
    print(f"\nR_major={R_major}, R_minor={R_minor}:")
    print(f"  Volume: {params['volume']:.2f}")
    print(f"  Surface area: {params['surface_area']:.2f}")
    print(f"  Aspect ratio: {params['aspect_ratio']:.2f}")
    print(f"  Inner radius: {params['inner_radius']:.2f}")
    print(f"  Outer radius: {params['outer_radius']:.2f}")

## 8. NASA's Analysis

White's work at NASA suggested that toroidal geometries might reduce energy requirements significantly. The key observations:

1. **Oscillating bubble thickness** can reduce energy by orders of magnitude
2. **Toroidal shape** concentrates exotic matter more efficiently
3. **Optimal aspect ratios** exist for minimum energy

These claims generated significant interest, though they remain controversial and subject to ongoing research.

In [ ]:
# Compare approximate energy requirements
print("Energy comparison (approximate):")
print("="*50)

alc = AlcubierreMetric(v0=1.0, R=2.0)
E_alc = alc.total_energy_estimate()
print(f"Alcubierre (R=2.0): E ~ {E_alc:.2e}")

for R_major, R_minor in [(2.0, 0.3), (2.0, 0.5), (2.0, 0.7)]:
    white = WhiteToroidalMetric(v0=1.0, R_major=R_major, R_minor=R_minor)
    # Rough estimate based on torus volume vs sphere volume
    V_torus = white.torus_parameters()['volume']
    V_sphere = (4/3) * np.pi * 2.0**3
    ratio = V_torus / V_sphere
    E_white = E_alc * ratio
    print(f"White (R_major={R_major}, R_minor={R_minor}): E ~ {E_white:.2e} (ratio: {ratio:.2f})")

## Summary

The White toroidal warp drive:

1. **[ASSUMPTION]** This implementation interprets White's AIAA presentations
2. **Toroidal geometry** concentrates exotic matter in a ring
3. **May reduce energy requirements** compared to spherical shells
4. **Passenger volume** is in the center of the torus
5. **Still requires exotic matter** - the fundamental issue persists
6. **Controversial claims** - energy reduction factors remain debated

The toroidal configuration represents an interesting geometric optimization approach, though the exact energy savings depend on specific details not fully published.